In [1]:
import os
import re
import fitz
import pandas as pd

In [3]:
PDF_DIR = r'G:\ACCOUNT PAYABLE\001 - INVOICES - FATTURE\LOGISTIC INVOICES\1- NEW\BRUZZONE DIRECT PAY DUTY STATEMENTS' 
PDFS    = [f for f in os.listdir(PDF_DIR) if f.startswith("DST") and f.endswith(".pdf")]
output_path = r'c:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Logistics Invoicing\CBP\Daily Statements_tool'

#  Entry row pattern – tolerant of new-lines
entry_rx = re.compile(
    r"""(?P<duty>\d{1,3}(?:,\d{3})*\.\d{2})\s+        # Estimated Duty
        0\.00\s+0\.00\s+0\.00\s+                      # three literal 0.00 columns
        (?P<fees>\d{1,3}(?:,\d{3})*\.\d{2})\s+        # User Fees / Interest
        (?P<total>\d{1,3}(?:,\d{3})*\.\d{2})\s+       # Total Amount
        (?P<entry>\d{3}[-–]\d+[-–]\d)\s+              # Entry number (dash or en-dash)
        (?P<ref>\d+)\s+                               # Ref number
        (?P<tp>\d{2}\s+\w+)                           # TP code (“03 BCA”)
    """,
    re.MULTILINE | re.VERBOSE,
)

date_rx  = re.compile(r"Process Date:\s*([0-9/]+)")

def money(txt: str) -> float:
    """'7,124.50' → 7124.50   '181.25' → 181.25"""
    return float(txt.replace(",", ""))

rows = []

for pdf in PDFS:
    path  = os.path.join(PDF_DIR, pdf)
    with fitz.open(path) as doc:
        text = "\n".join(p.get_text() for p in doc)

    # Extract Process Date once per file
    md = date_rx.search(text)
    proc_date = md.group(1) if md else ""

    # Find every entry block (some PDFs have 2+ entries)
    for m in entry_rx.finditer(text):
        rows.append({
            "Statement"          : pdf[:-4],                 # file name minus .pdf
            "Process Date"       : proc_date,
            "Entry Number"       : m["entry"],
            "Ref Number"         : m["ref"],
            "TP"                 : m["tp"].replace("\n", " "),
            "Estimated Duty"     : money(m["duty"]),
            "User Fees/Interest" : money(m["fees"]),
            "Total Amount"       : money(m["total"]),
        })

# → DataFrame + Excel
df = pd.DataFrame(rows)
print(df)                              # sanity-check in the console
outfile = os.path.join(output_path, "dst_statements.xlsx")
df.to_excel(outfile, index=False)
print(f"\n✓ {len(df)} rows written to {outfile}")

         Statement Process Date   Entry Number Ref Number      TP  \
0    DST042519902D      7/18/25  237-3171276-8    3171276  03 BCA   
1    DST132521001R      7/29/25  237-3171188-5    3171188  03 BCA   
2    DST132521001R      7/29/25  237-3171262-8    3171262  03 BCA   
3    DST132521001R      7/29/25  237-3171273-5    3171273  03 BCA   
4    DST1325211025      7/30/25  237-3171282-6    3171282  03 BCA   
..             ...          ...            ...        ...     ...   
113  DST522521103M      7/30/25  237-3171528-2    3171528  03 BCA   
114  DST532519804E      7/17/25  237-3170964-0    3170964  03 BCA   
115  DST532519804E      7/17/25  237-3170967-3    3170967  03 BCA   
116  DST532521105G      7/30/25  237-3171181-0    3171181  03 BCA   
117  DST532521105G      7/30/25  237-3171268-5    3171268  03 BCA   

     Estimated Duty  User Fees/Interest  Total Amount  
0            2482.7              117.04       2599.74  
1            3614.2              170.37       3784.57  
2  

In [4]:

# Load Bruzzone combined customs file 
bruzzone_file = r'c:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Logistics Invoicing\Bruzzone\2025-YTD-Consolidated-Invoices for CBP.xlsx'
df_customs = pd.read_excel(bruzzone_file)

# Extract base reference from 'Bruzzone Invoice No.'
df_customs['Ref Number'] = df_customs['Bruzzone Invoice No.'].astype(str).str.extract(r'(^\d+)')

# Select only the PO columns
po_cols = [col for col in df_customs.columns if col.startswith('Customer Ref')]

# Melt/unpivot to stack POs vertically
df_pivoted = df_customs[['Ref Number'] + po_cols].melt(
    id_vars= 'Ref Number',
    value_vars = po_cols,
    value_name = 'PO Number'
)

# Drop blanks and duplicates
df_cleaned = df_pivoted.dropna(subset=['PO Number']).drop_duplicates()

# Reset index for a clean final table
df_cleaned = df_cleaned[['Ref Number', 'PO Number']].reset_index(drop=True)

print (df_cleaned.head())

  Ref Number   PO Number
0    3166472  4500195790
1    3166474  4500200541
2    3166512  4500200436
3    3166607  4500200323
4    3166609  4500200831


In [8]:
# Merge CBP and Bruzzone for PO info
df_merge = df.merge(df_cleaned, on='Ref Number', how='left')
df_merge.columns
df_merge = df_merge[['Statement', 'Process Date', 'Entry Number', 'Ref Number', 'PO Number', 'Estimated Duty', 'User Fees/Interest', 'Total Amount']]
df_merge.to_excel('test.xlsx', index=False)